In [1]:
import pandas as pd

In [2]:
path_trans = (r"C:\Users\ASUS\Documents\Project\Momo case study\Data\processed\df_transactions.csv")
path_users = (r"C:\Users\ASUS\Documents\Project\Momo case study\Data\processed\df_users.csv")
path_products = (r"C:\Users\ASUS\Documents\Project\Momo case study\Data\processed\df_products.csv")

In [40]:
df_products = pd.read_csv(path_products, encoding = 'utf-8')
df_users = pd.read_csv(path_users, encoding = 'utf-8')
df_trans = pd.read_csv(path_trans, encoding = 'utf-8' )


In [22]:
df_trans.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13495 entries, 0 to 13494
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   user_id          13495 non-null  int64  
 1   order_id         13495 non-null  int64  
 2   Date             13495 non-null  object 
 3   Amount           13495 non-null  float64
 4   Merchant_id      13495 non-null  int64  
 5   Purchase_status  13495 non-null  int64  
dtypes: float64(1), int64(4), object(1)
memory usage: 632.7+ KB


In [23]:
df_trans = df_trans.merge(
    df_products, on="Merchant_id", how="left"
)

df_trans["Revenue"] = df_trans["Amount"] * df_trans["Rate_pct"]

In [24]:
jan_2020_revenue = df_trans.loc[
    (df_trans['Date'] >= "2020-01-01") &
    (df_trans['Date'] < "2021-01-01"), 
    "Revenue"].sum()


In [25]:
print(jan_2020_revenue)

1871272702.0


In [26]:
df_trans['Date'] = pd.to_datetime(df_trans['Date'])


In [27]:
df_users['First_tran_date'] = pd.to_datetime(df_users['First_tran_date'])


In [41]:
print(df_users['First_tran_date'].isnull().sum())

74


In [28]:
most_profitable_month = (
    df_trans
    .groupby(df_trans["Date"].dt.to_period("M"))["Revenue"]
    .sum()
    .idxmax()
)
total_for_month = (
    df_trans
    .groupby(df_trans['Date'].dt.to_period("M"))["Revenue"]
    .sum()
    .max()
)
print(most_profitable_month,total_for_month)

2020-09 170220000.0


In [29]:
weekday_avg = (
    df_trans
    .groupby(df_trans["Date"].dt.day_name())["Revenue"]
    .mean()
)
most_profitable_day = weekday_avg.idxmax()
least_profitable_day = weekday_avg.idxmin()

print(most_profitable_day, least_profitable_day)

Wednesday Monday


In [30]:
df_users.rename(columns={'User_id': 'user_id'}, inplace=True)

In [31]:
df_users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13428 entries, 0 to 13427
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   user_id          13428 non-null  int64         
 1   First_tran_date  13354 non-null  datetime64[ns]
 2   Location         13428 non-null  object        
 3   Age              13428 non-null  object        
 4   Gender           13428 non-null  object        
dtypes: datetime64[ns](1), int64(1), object(3)
memory usage: 524.7+ KB


In [32]:
df_trans.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13495 entries, 0 to 13494
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   user_id          13495 non-null  int64         
 1   order_id         13495 non-null  int64         
 2   Date             13495 non-null  datetime64[ns]
 3   Amount           13495 non-null  float64       
 4   Merchant_id      13495 non-null  int64         
 5   Purchase_status  13495 non-null  int64         
 6   Merchant_name    13495 non-null  object        
 7   Rate_pct         13495 non-null  int64         
 8   Revenue          13495 non-null  float64       
dtypes: datetime64[ns](1), float64(2), int64(5), object(1)
memory usage: 949.0+ KB


In [33]:
df_trans_tx = df_trans.merge(
    df_users,
    on="user_id",
    how="left"
)

In [36]:
df_trans_tx.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13576 entries, 0 to 13575
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   user_id          13576 non-null  int64         
 1   order_id         13576 non-null  int64         
 2   Date             13576 non-null  datetime64[ns]
 3   Amount           13576 non-null  float64       
 4   Merchant_id      13576 non-null  int64         
 5   Purchase_status  13576 non-null  int64         
 6   Merchant_name    13576 non-null  object        
 7   Rate_pct         13576 non-null  int64         
 8   Revenue          13576 non-null  float64       
 9   First_tran_date  13502 non-null  datetime64[ns]
 10  Location         13576 non-null  object        
 11  Age              13576 non-null  object        
 12  Gender           13576 non-null  object        
dtypes: datetime64[ns](2), float64(2), int64(5), object(4)
memory usage: 1.3+ MB


In [39]:
print(df_trans_tx['First_tran_date'].isnull().sum())

74


In [37]:
df_trans_tx["Age"] = df_trans_tx["Age"].astype("category")
df_trans_tx["Age"].value_counts()

Age
23_to_27    3452
28_to_32    2829
unknown     2541
33_to_37    1774
18_to_22    1706
>37         1274
Name: count, dtype: int64

In [ ]:
df_trans_tx["Type_user"] = (
    df_trans_tx["Date"].dt.to_period("M") ==
    df_trans_tx["First_tran_date"].dt.to_period("M")
)

df_trans_tx["Type_user"] = df_trans["Type_user"].map({
    True: "New",
    False: "Current"
})

In [20]:
new_users_dec_2020 = (
    df_trans[
        (df_trans["Type_user"] == "New") &
        (df_trans["Date"].dt.to_period("M") == "2020-12")
    ]["user_id"]
    .nunique()
)

new_users_dec_2020

72